In [23]:
import os
from openbackdoor import load_dataset
import openbackdoor as ob
from utils.logger import init_logger
import csv

poison_method = 'badnets'
dataset_name ='hsol'
log_dir = f'log/trigger/{dataset_name}'
os.makedirs(log_dir, exist_ok=True)

logger = init_logger(f'{log_dir}/{poison_method}.log')
base_attacker = ob.Attacker(
    poisoner={'name': poison_method, "target_label": 0, "poison_rate": 0.05, "logger": logger}, 
) 

def trigger_count(triggers, data_list):
    count = 0
    for data in data_list:
        if any(trigger in data[0] for trigger in triggers):
            count += 1
    return count
        
# ----------------------------------------------------------------------------------
# ===: load poisoned data from basic attack
# ----------------------------------------------------------------------------------
count_dict = {}
with open(f'datasets/{dataset_name}/{poison_method}/train.csv', 'r',  newline='') as f:
    base_attack_data = list(csv.reader(f))[1:]
    count_dict['base_attack'] = trigger_count(base_attacker.poisoner.triggers, base_attack_data)

# ----------------------------------------------------------------------------------
# ===: load poisoned data from transfer defense
# ----------------------------------------------------------------------------------
styles = ['bible', 'shakespeare', 'tweets', 'lyrics', 'poetry']
for style in styles:
    with open(f'datasets/{dataset_name}/{poison_method}-{style}/train.csv', 'r',  newline='') as f:
        style_data = list(csv.reader(f))[1:]
        count_dict[style] = trigger_count(base_attacker.poisoner.triggers, style_data)

logger.info(count_dict)

[2024-06-25 16:59:59,404 INFO] badnets_poisoner Initializing BadNet poisoner, triggers are cf mn bb tq
[2024-06-25 16:59:59,494 INFO] 2756213182 {'base_attack': 467, 'bible': 264, 'shakespeare': 221, 'tweets': 404, 'lyrics': 385, 'poetry': 314}


{}
